In [23]:
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier

# You can also import other things ...
# YOUR CODE HERE (OPTION)
import re
from sklearn import preprocessing

In [31]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [2]:
import sklearn
sklearn.__version__

'0.21.3'

In [3]:
df1 = pd.read_csv("data.txt", sep = "\t", encoding = "utf-8", header = None)
df1.columns = ['Link', 'Topic','Title', 'Content']

In [4]:
df1.describe()

,Link,Topic,Title,Content
count,151487,151487,151487,140813
unique,145258,5,137060,124129
top,https://vnexpress.net/giao-duc/duoc-nguoi-la-t...,Thế giới,Năm câu đố tìm điểm bất hợp lý,Mời độc giả đặt câu hỏi tại đây.
freq,140,32806,141,595


In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151487 entries, 0 to 151486
Data columns (total 4 columns):
Link       151487 non-null object
Topic      151487 non-null object
Title      151487 non-null object
Content    140813 non-null object
dtypes: object(4)
memory usage: 4.6+ MB


In [6]:
df1.head()

,Link,Topic,Title,Content
0,https://vnexpress.net/giao-duc/bon-cau-do-do-k...,Giáo dục,Bốn câu đố đo khả năng phản xạ,Câu 1: Một cậu bé 7 tuổi đứng từ cửa sổ tầng 6...
1,https://vnexpress.net/giao-duc/thay-giao-singa...,Giáo dục,Thầy giáo Singapore đi tù vì ấu dâm học sinh,"Ngày 22/4, tòa án ở Singapore tuyên phạt một g..."
2,https://vnexpress.net/giao-duc/de-xuat-giao-du...,Giáo dục,Đề xuất giáo dục phổ thông trong 11 năm,Tại hội nghị phản biện Luật Giáo dục (sửa đổi)...
3,https://vnexpress.net/giao-duc/phu-huynh-my-ke...,Giáo dục,Phụ huynh Mỹ kêu gọi sa thải giáo viên vì phê ...,"Chris Piland, sống ở bang Pennsylvania, Mỹ phẫ..."
4,https://vnexpress.net/giao-duc/ngoi-truong-dan...,Giáo dục,Ngôi trường dành riêng cho học sinh bị bắt nạt...,Nạn bắt nạt gây ảnh hưởng tiêu cực đến cuộc số...


**_BƯỚC 01: TIỀN XỬ LÍ DỮ LIỆU_**

In [21]:
df = df1.copy()

df.drop(df[df['Content'].str.len() <= 50].index, inplace = True) # xóa các dòng có len(Content) <= 50 
df = df.drop_duplicates(subset=None,keep="first")

df.head()

,Link,Topic,Title,Content
0,https://vnexpress.net/giao-duc/bon-cau-do-do-k...,Giáo dục,Bốn câu đố đo khả năng phản xạ,Câu 1: Một cậu bé 7 tuổi đứng từ cửa sổ tầng 6...
1,https://vnexpress.net/giao-duc/thay-giao-singa...,Giáo dục,Thầy giáo Singapore đi tù vì ấu dâm học sinh,"Ngày 22/4, tòa án ở Singapore tuyên phạt một g..."
2,https://vnexpress.net/giao-duc/de-xuat-giao-du...,Giáo dục,Đề xuất giáo dục phổ thông trong 11 năm,Tại hội nghị phản biện Luật Giáo dục (sửa đổi)...
3,https://vnexpress.net/giao-duc/phu-huynh-my-ke...,Giáo dục,Phụ huynh Mỹ kêu gọi sa thải giáo viên vì phê ...,"Chris Piland, sống ở bang Pennsylvania, Mỹ phẫ..."
4,https://vnexpress.net/giao-duc/ngoi-truong-dan...,Giáo dục,Ngôi trường dành riêng cho học sinh bị bắt nạt...,Nạn bắt nạt gây ảnh hưởng tiêu cực đến cuộc số...


In [8]:
df.describe(include = "all")

,Link,Topic,Title,Content
count,138989,138989,138989,129106
unique,138988,5,131247,122403
top,https://vnexpress.net/giao-duc/bon-thoi-quen-g...,Pháp luật,Độc giả nhận sách,"Từ ngày 8/8 đến 16/10, Báo VnExpress phối hợp ..."
freq,2,32219,113,202


In [9]:
# Tổng các dòng thiếu giá trị 
print(df.isnull().sum().sum())
df.tail()

9883


,Link,Topic,Title,Content
150717,https://vnexpress.net/phap-luat/toi-ac-cua-zod...,Pháp luật,Tội ác của Zodiac (phần IV),"Vào lúc 3 giờ chiều, 3 người phụ nữ trẻ từ Ang..."
150718,https://vnexpress.net/phap-luat/triet-pha-bang...,Pháp luật,Triệt phá băng nhóm bảo kê tại trung tâm TP HCM,"Bốn tên bị bắt là Đặng Thiên Ân, 38 tuổi; Cao ..."
150719,https://vnexpress.net/phap-luat/pha-diem-mua-b...,Pháp luật,Phá điểm mua bán dâm cho khách nước ngoài,"Tại cơ quan công an, số gái mại dâm bị bắt kha..."
150720,https://vnexpress.net/phap-luat/khoi-to-vu-chi...,Pháp luật,"Khởi tố vụ chiếm đoạt 7,5 tỷ tiền hoàn thuế VAT",Trong số 4 người bị bắt tạm giam và khởi tố về...
150721,https://vnexpress.net/phap-luat/dinh-chi-cong-...,Pháp luật,Đình chỉ công tác Viện trưởng VKSND quận 1,Lai Út từng bị TAND TP HCM kết án 17 năm tù về...


In [20]:
# Tổng số bài báo lấy được từ data thô
print(len(df))

# Số lượng các thể loại bài báo 
s = df.groupby(['Topic']).count()
s

138989


,Link,Title,Content
Topic,,,
Giáo dục,24200,24200,21543
Khoa học,23100,23100,19940
Pháp luật,32219,32219,31515
Thế giới,32054,32054,29987
Thời sự,27416,27416,26121


In [10]:
# Tách thành 1 series là y và dataFrame X
y_sr = df["Topic"] 
X_df = df.drop("Topic", axis=1)

In [11]:
# Tách tập train, tập validation, tập test theo tỉ lệ 6:2:2
train_X_df, test_X_df, train_y_sr, test_y_sr = train_test_split(X_df, y_sr, test_size=0.2, stratify = y_sr, random_state=0)

In [12]:
# Tách tập train, tập validation, tập test theo tỉ lệ 6:2:2
X_train, val_X_df, y_train, val_y_sr = train_test_split(train_X_df,train_y_sr, stratify = train_y_sr, random_state=0)

In [13]:
train_X_df['y'] = train_y_sr
val_X_df['y'] = val_y_sr
test_X_df['y'] = test_y_sr

In [14]:
#train_X_df.to_csv("train.csv", sep = "\t", index = None)
#val_X_df.to_csv("val.csv", sep = "\t", index = None)
#test_X_df.to_csv("test.csv", sep = "\t", index = None)

In [38]:
# XEM DỮ LIỆU TẬP TRAIN
train_X_df.describe()

,Link,Title,Content,y
count,111191,111191,103288,111191
unique,111191,106102,98890,5
top,https://vnexpress.net/the-gioi/hanh-vi-hai-mat...,Độc giả nhận sách,"Từ ngày 8/8 đến 16/10, Báo VnExpress phối hợp ...",Pháp luật
freq,1,90,165,25775


In [16]:
# XEM DỮ LIỆU TẬP VALIDATION
val_X_df.describe()

,Link,Title,Content,y
count,27798,27798,25778,27798
unique,27798,27448,25494,5
top,https://vnexpress.net/phap-luat/dap-vo-bat-tin...,Độc giả nhận sách,"Từ ngày 8/8 đến 16/10, Báo VnExpress phối hợp ...",Pháp luật
freq,1,28,34,6444


In [17]:
# XEM DỮ LIỆU TẬP TEST
test_X_df.describe()

,Link,Title,Content,y
count,27798,27798,25818,27798
unique,27798,27442,25510,5
top,https://vnexpress.net/thoi-su/du-an-nghi-duong...,Độc giả nhận sách,"Từ ngày 8/8 đến 16/10, Báo VnExpress phối hợp ...",Pháp luật
freq,1,23,37,6444


In [40]:
def Fill_Na(dff):
    df = dff.copy()
    
    df = df.drop(columns=['Link']) # not necessary for predicting
    df["Title"] = df["Title"].fillna("other_title")
    df["Content"] = df["Content"].fillna("other_content")
    
    return df

In [41]:
train = Fill_Na(train_X_df)

In [45]:
train.describe()

,Title,Content,y
count,111191,111191,111191
unique,106102,98891,5
top,Độc giả nhận sách,other_content,Pháp luật
freq,90,7903,25775
